In [1]:
# default_exp facade


In [11]:
#export
import random
from nbdev.showdoc import *
#from ds4se.mgmnt.prep.bpe import *
#from ds4se.exp.info import *
#from ds4se.desc.stats import *
import pandas as pd
import sentencepiece as sp
from pathlib import Path
from collections import Counter
from ds4se.mining.unsupervised.traceability.eval import *
from enum import Enum, unique, auto
import os

In [10]:
#export
@unique
class LinkType(Enum):
    req2tc = auto()
    req2src = auto()

In [8]:
#export
def TraceLinkValue(source, target, technique, word2vec_metric = "WMD"):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #param technique what tecchnique to use to calculate trace values
    #return a trace value between [0,1]
    
    #TODO make the string saved and then return the path to that string
    sourcePath = "path/to/something"#this is where we save the passed in source
    targetPath = "./something"
    
    value = random.randint(0,1)/100

    
    if (technique == "VSM"):
        pass
    if (technique == "LDA"):
        pass
    if (technique == "orthogonal"):
        pass
    if (technique == "LSA"):
        pass
    if (technique == "JS"):
        pass
    if (technique == "word2vec"):
        parameter = {
            "vectorizationType": VectorizationType.word2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": 'test_data/models/word2vec_libest.model',
            "source_path": 'val.csv',
            "target_path": 'test_data/val.csv',
            "system_path": 'test_data/val.csv',
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        word2vec = Word2VecSeqVect(parameter)
        word2vec.df_source = source_df
        word2vec.df_target = target_df
        links = [(source_df["ids"][0],target_df["ids"][0])]
        if (word2vec_metric == "SCM"):
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.SCM])
        else:
            computeDistanceMetric = word2vec.computeDistanceMetric(links, metric_list = [DistanceMetric.WMD])
        value = (computeDistanceMetric[0][0][2],computeDistanceMetric[0][0][3])    
        
    if (technique == "doc2vec"):
        parameter = {
            "vectorizationType": VectorizationType.doc2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": 'test_data/models/doc2vec_libest.model',
            "source_path": 'test_data/val.csv',
            "target_path": 'test_data/val.csv',
            "system_path": 'test_data/val.csv',
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        
        source_df = pd.DataFrame({ "ids": ["source"],  "text":[source]})
        target_df = pd.DataFrame({ "ids": ["target"],  "text":[target]})
        doc2vec = Doc2VecSeqVect(params = parameter)
        doc2vec.df_source = source_df
        doc2vec.df_target = target_df
        doc2vec.InferDoc2Vec(steps=200)
        table = doc2vec.ComputeDistanceArtifacts( sampling=True, samples = 50, metric_list = [DistanceMetric.EUC] )
        value = (table[0][0][2], table[0][0][3])
        #The bottom is here for reference -- may not need it
#         doc2vec.SaveLinks()
#         #will most likely need to change this part need to change this part to a different path
#         path_to_ground_truth = '/tf/main/benchmarking/traceability/testbeds/groundtruth/english/[libest-ground-req-to-tc].txt'
#         doc2vec.MatchWithGroundTruth(path_to_ground_truth)
#         doc2vec.SaveLinks(grtruth = True)
#         #TODO find logic to LoadLink properly and display what is needed
    
    return value

#export
def NumDoc(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    difference = source_doc - target_doc
    return [source_doc, target_doc, difference, -difference]
   
#export
def VocabSize(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files in terms of vocab
    source_list = preprocess(source)
    target_list = preprocess(target)
    source_size = len(source_list[0])
    target_size = len(target_list[0])
    difference = source_size - target_size
    return [source_size, target_size, difference, -difference]  

#export
def AverageToken(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files in terms of tokens
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    
    source_list = preprocess(source)
    target_list = preprocess(target)
    
    source_total_token = sum(source_list[0].values())
    target_total_token = sum(target_list[0].values())

    source_token = source_total_token/source_doc
    target_token = target_total_token/target_doc
    difference = source_token - target_token
    return [source_token, target_token, difference, -difference]

#export
def Vocab(artifacts_df):
    #we can add a parameter for user to specify the number of most frequent token to return
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return a list containing the the difference between the two files in terms of vocab
    cnts = preprocess(artifacts_df)
    vocab_list = cnts[0].most_common(3)
    total = sum(cnts[0].values())
    vocab_dict = dict()
    vocab_dict[vocab_list[0][0]] = [vocab_list[0][1], vocab_list[0][1]/total]
    vocab_dict[vocab_list[1][0]] = [vocab_list[1][1], vocab_list[1][1]/total]
    vocab_dict[vocab_list[2][0]] = [vocab_list[2][1], vocab_list[2][1]/total]

    return vocab_dict
    
    
#export
def VocabShared(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the similarities of vocab in the files 
    df = pd.concat([source, target])
    return Vocab(df) 
    
#export
def SharedVocabSize(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the similarities of vocab sizes in the files 
    df = pd.concat([source, target])
    df_counts = preprocess(df)
    shared_size = len(df_counts[0])
    return shared_size

#export
def MutualInformation(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the mutual information 
    mutual_information = random.randint(100,200)
    return mutual_information
    
#export
def CrossEntropy(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the entropy
    cross_entropy = random.randint(100,200)
    cross_entropy = get_system_entropy_from_df(source, "col1",)
    return cross_entropy
    
#export
def KLDivergence(source, target):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the divergence  
    divergence = random.randint(100,200)
    return divergence

#export
def get_docs(df, spm):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the document
    docs = []
    for fn in df["col1"]:
        docs += spm.EncodeAsPieces(fn)
    return docs

#export
def get_counters(docs):
    #param source a string of the entire source file  
    #param target a string of the entire target file 
    #return the number of tokens   
    doc_cnts = []
    cnt = Counter()
    for tok in docs:
        cnt[tok] += 1 
        doc_cnts.append(cnt)
    return doc_cnts

#export
def preprocess(artifacts_df):
    spm = sp.SentencePieceProcessor()
    output = Path('test_data\models')
    system_name = "test"
    spm.Load(f"{output / system_name}.model")
    docs = get_docs(artifacts_df,spm)
    cnts = get_counters(docs)
    return cnts

In [7]:
str1 = open('test_data/LibEST_semeru_format/requirements/RQ11.txt', 'r').read()
str2 = open('test_data/LibEST_semeru_format/test/us4020.c', 'r').read()
TraceLinkValue(str1,str2,"word2vec")

2020-10-18 20:20:54,480 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-10-18 20:20:54,489 : INFO : built Dictionary(1815 unique tokens: ['@return', 'Converts', 'The', 'a', 'and']...) from 153 documents (total 5769 corpus positions)
2020-10-18 20:20:54,490 : INFO : loading Word2Vec object from test_data/models/word2vec_libest.model
2020-10-18 20:20:54,548 : INFO : loading wv recursively from test_data/models/word2vec_libest.model.wv.* with mmap=None
2020-10-18 20:20:54,549 : INFO : setting ignored attribute vectors_norm to None
2020-10-18 20:20:54,550 : INFO : loading vocabulary recursively from test_data/models/word2vec_libest.model.vocabulary.* with mmap=None
2020-10-18 20:20:54,551 : INFO : loading trainables recursively from test_data/models/word2vec_libest.model.trainables.* with mmap=None
2020-10-18 20:20:54,552 : INFO : setting ignored attribute cum_table to None
2020-10-18 20:20:54,553 : INFO : loaded test_data/models/word2vec_libest.model
2020-10-18 20:20:5

(0.5401013460499512, 0.6493079189657213)

In [93]:
# nbdev_build_docs
# notebook2script()

In [ ]:
#FacadeTest
#Test case for proto TLV

#Prototype should print a value between 0 or 1. Input values aren't used
testTLV = TraceLinkValue(0,1,1)
assert((testTLV == 0) or (testTLV == 1))

In [ ]:
#FacadeTest
#Test Case for Proto NumDoc

#Prototype should print an array based on the randomly generated numbers. Input values aren't used.
testND = NumDoc(100,10000000)
assert(isinstance(testND, list))

In [ ]:
#FacadeTest
#Test case for Vocab size

#Prototype should return an array based on random number values. Inputs aren't used.
testVS = VocabSize(100,100000)
assert(isinstance(testVS, list))

In [ ]:
#FacadeTest
#Test Case for AverageToken

#Prototype should return an array/list. Inputs aren't used
testAT = AverageToken(100,100)
assert(isinstance(testAT, list))

In [ ]:
#FacadeTest
#Test Case for Vocab

#Prototype Vocab should return dictionary type. 
#Will expand to test dictionary values once presets can be generated and tested
testVocab = Vocab("file")

assert(isinstance(testVocab,dict))

In [ ]:
#FacadeTest
#Test VocabShared

#Prototype should show a dictionary
#Test will be expanded to verify frequency verification in two samples
testVocabS = VocabShared("source","target")
assert(isinstance(testVocabS,dict))


In [ ]:
#FacadeTest
#Test Shared vocab size

#Prototype should return an i
testSVS = SharedVocabSize("source","target")
assert(testSVS in range(100,200))

In [ ]:
#FacadeTest
#TEst Mutual information

#Proto mutual information should get an int
testMutualInfo = MutualInformation("source","target")
assert(testMutualInfo in range(100,200))

In [ ]:
#FacadeTest
#Test Cross Entropy

#Proto cross Entropy should return an int
testCrossEntropy = CrossEntropy("Sauce","Target")
assert(testCrossEntropy in range(100,200))

#Eventually cross entropy will properly implement Entropy calculation and the test will compare a known/predicted entropy
# to a calculated value


In [ ]:
#FacadeTest
#Test KLDivergence

#Proto KLDivegence will return an random int
testKLDiver = KLDivergence("source","target")
assert(testKLDiver in range(100,200))